In [ ]:
!pip install --upgrade pip setuptools
!pip install numpy==1.19.1
!pip install -r /content/hovernet/requirements.txt

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 71.0.4
    Uninstalling setuptools-71.0.4:
      Successfully uninstalled setuptools-71.0.4
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 24.6 MB/s eta 0:00:00
ERROR: Operation cancelled by user
  Installing build dependencies ... canceledERROR: Could not open requirements file: [Errno 2] No such file or directory: '/content/hovernet/requirements.txt'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install docopt
!pip install geojson
!pip install tensorboardX
import tensorboardX
!pip install tqdm

  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=12ed59a13a3bea5d002b4bb3c0e8a6802be8ceae981f958c3b6d0bed9ae7808f
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
output_dir = '/content/drive/My Drive/output'
import os
os.makedirs(output_dir, exist_ok=True)
!python hover_net-master/run_infer.py --gpu=0 --nr_types=6 --type_info_path=hover_net-master/type_info.json --model_path=hover_net-master/models/hovernet_fast_pannuke_type_tf2pytorch.tar --model_mode=fast tile --input_dir=hover_net-master/HE_old_40X --output_dir='/content/drive/My Drive/output' --save_qupath

|2024-07-11|20:59:23.870| [INFO] .... Detect #GPUS: 1
Process Patches: 136it [08:44,  3.86s/it]
|2024-07-11|21:09:04.073| [INFO] ........................ Done Assembling D-3
|2024-07-11|21:10:12.160| [INFO] ........................ Done Assembling L-2
|2024-07-11|21:10:35.629| [INFO] ........................ Done Assembling B-8
|2024-07-11|21:10:43.933| [INFO] ........................ Done Assembling P-1
|2024-07-11|21:10:59.896| [INFO] ........................ Done Assembling L-8


In [ ]:
import sys
sys.argv = ['script_name.py', '/content/B-8.json', '/content/B-8.geojson']

import argparse
import gzip
import json
from pathlib import Path
from shapely.geometry import Polygon

# Type mapping dictionary for HoverNet output
type_mapping = {
    0: "nolabel",
    1: "neoplastic",
    2: "inflammatory",
    3: "connective",
    4: "necrosis",
    5: "non-neoplastic",
}

def hovernet_to_feature_dict(instance_id: int, d: dict):
    """Convert HoverNet instance dictionary to GeoJSON feature dictionary."""
    poly = Polygon(d["contour"])
    coordinates = [list(coord) for coord in poly.exterior.coords]  # Convert to list for JSON serialization
    return {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [coordinates],
        },
        "properties": {
            "classification": type_mapping[d["type"]],
            "instance_id": int(instance_id),
            "type_str": type_mapping[d["type"]],
            "type_int": d["type"],
            "type_prob": d["type_prob"],
        },
    }

def hovernet_to_geojson(d: dict):
    """Convert HoverNet output dictionary to GeoJSON format."""
    assert "nuc" in d.keys(), "expected 'nuc' key"
    assert "mag" in d.keys(), "expected 'mag' key"
    #assert d["mag"] == 40, "this script was designed for mag=40x"
    features = [hovernet_to_feature_dict(ii, dd) for ii, dd in d["nuc"].items()]
    return {
        "type": "FeatureCollection",
        "features": features,
    }

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description=__doc__)
    parser.add_argument("input", help="JSON file with HoverNet predictions.")
    parser.add_argument("output", help="Output GeoJSON file")
    args = parser.parse_args()

    input_path = Path(args.input)
    output_path = Path(args.output)

    if not input_path.exists():
        raise FileNotFoundError(f"Input file not found: {args.input}")
    if output_path.exists():
        raise FileExistsError(f"Output file exists: {args.output}")

    print("Reading HoverNet polygons...")
    open_fn = gzip.open if args.input.endswith(".gz") else open
    with open_fn(args.input) as f:
        d = json.load(f)

    print("Converting to GeoJSON...")
    geojson = hovernet_to_geojson(d)

    print(f"Writing output to {args.output}")
    open_fn = gzip.open if args.output.endswith(".gz") else open
    with open_fn(args.output, "wt") as f:
        json.dump(geojson, f)

    print("Done.")
    from google.colab import files
    files.download('B-8.geojson')
    files.download('D-3.geojson')
    files.download('L-2.geojson')
    files.download('L-8.geojson')
    files.download('P-1.geojson')


Reading HoverNet polygons...
Converting to GeoJSON...
Writing output to /content/B-8.geojson
Done.


In [ ]:
!unzip '/content/drive/MyDrive/hovernet.zip'

Archive:  /content/drive/MyDrive/hovernet.zip
  inflating: hovernet/.gitignore     
   creating: hovernet/.vscode/
  inflating: hovernet/.vscode/launch.json  
 extracting: hovernet/__init__.py    
   creating: hovernet/__pycache__/
  inflating: hovernet/__pycache__/config.cpython-311.pyc  
  inflating: hovernet/__pycache__/convert_format.cpython-311.pyc  
  inflating: hovernet/__pycache__/dataset.cpython-311.pyc  
  inflating: hovernet/CHANGELOG.md   
   creating: hovernet/checkpoints/
  inflating: hovernet/compute_stats.py  
  inflating: hovernet/config.py      
  inflating: hovernet/convert_chkpt_tf2pytorch.py  
  inflating: hovernet/convert_format.py  
   creating: hovernet/data/
   creating: hovernet/data/images/
  inflating: hovernet/data/images/B-8.tiff  
  inflating: hovernet/data/images/D-3.tiff  
  inflating: hovernet/data/images/L-2.tiff  
  inflating: hovernet/data/images/L-8.tiff  
  inflating: hovernet/data/images/P-1.tiff  
   creating: hovernet/data/labels/
  inflating: 

In [ ]:
import scipy.io
import numpy as np

# Load the .mat file
mat_contents = scipy.io.loadmat('oldb8.mat')

# Print each variable's name and its contents
for key, value in mat_contents.items():
    if not key.startswith('__'):  # This avoids internal metadata fields
        print(f"Variable: {key}")
        print(value)
        print(value.shape)
        print(np.unique(value))
        index_of_one = np.where(value == 1)[0]
        print(index_of_one)
        print()  # Add a blank line for readability

Variable: inst_map
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(4972, 4972)
[   0    1    2 ... 6674 6675 6676]
[278 278 278 278 278 278 278 279 279 279 279 279 279 279 279 279 279 279
 279 280 280 280 280 280 280 280 280 280 280 280 280 280 280 281 281 281
 281 281 281 281 281 281 281 281 281 281 281 281 281 282 282 282 282 282
 282 282 282 282 282 282 282 282 282 282 282 282 283 283 283 283 283 283
 283 283 283 283 283 283 283 283 283 283 283 283 284 284 284 284 284 284
 284 284 284 284 284 284 284 284 284 284 284 284 284 285 285 285 285 285
 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285 286 286 286
 286 286 286 286 286 286 286 286 286 286 286 286 286 286 286 286 286 287
 287 287 287 287 287 287 287 287 287 287 287 287 287 287 287 287 287 287
 287 287 288 288 288 288 288 288 288 288 288 288 288 288 288 288 288 288
 288 288 288 288 288 289 289 289 289 289 289 289 289 289 289 289 289 289
 289 289 

In [ ]:
import numpy as np
import scipy.io
from shapely.geometry import Polygon, Point
import json
from tqdm import tqdm
import os

def create_mask_from_polygon(polygon, bounds, shape):
    minx, miny, maxx, maxy = bounds
    mask = np.zeros(shape, dtype=bool)

    for y in range(int(miny), int(maxy) + 1):
        for x in range(int(minx), int(maxx) + 1):
            if polygon.contains(Point(x, y)):
                mask[y-int(miny), x-int(minx)] = True

    return mask

def extract_coordinates(geometry):
    if geometry['type'] == 'Polygon':
        return geometry['coordinates'][0]
    elif geometry['type'] == 'MultiPolygon':
        return geometry['coordinates'][0][0]
    else:
        raise ValueError(f"Unsupported geometry type: {geometry['type']}")

def process_geojson(file_name, image_shape):
    # Load the GeoJSON file
    with open(file_name) as f:
        geojson_data = json.load(f)

    # Initialize mask
    inst_map = np.zeros(image_shape, dtype=np.int32)
    type_map = np.zeros(image_shape, dtype=np.int32)
    inst_type = []
    inst_uiud = []

    # Keep track of unique nuclei
    unique_nuclei = set()

    total_features = len(geojson_data['features'])
    valid_nuclei = 0
    duplicate_nuclei = 0
    skipped_nuclei = 0

    # Process features with a progress bar
    for feature in tqdm(geojson_data['features'], desc=f"Processing {file_name}", total=total_features):
        try:
            coordinates = extract_coordinates(feature['geometry'])
            polygon = Polygon(coordinates)

            if not polygon.is_valid:
                skipped_nuclei += 1
                continue

            # Check if this nucleus has already been processed
            centroid = polygon.centroid
            centroid_key = (round(centroid.x, 2), round(centroid.y, 2))

            if centroid_key in unique_nuclei:
                duplicate_nuclei += 1
                continue

            unique_nuclei.add(centroid_key)

            bounds = polygon.bounds

            # Create a mask for the bounding box of the polygon
            minx, miny, maxx, maxy = map(int, bounds)
            mask = create_mask_from_polygon(polygon, bounds, (maxy-miny+1, maxx-minx+1))

            # Update instance map
            inst_map[miny:maxy+1, minx:maxx+1][mask] = valid_nuclei + 1

            # Update type and UID
            type_val = string_to_number.get(feature['properties']['classification']['name'], 0)
            type_map[miny:maxy+1, minx:maxx+1][mask] = type_val
            inst_type.append([type_val])
            inst_uiud.append([valid_nuclei + 1])

            valid_nuclei += 1
        except Exception as e:
            print(f"Error processing a feature in {file_name}: {str(e)}")
            skipped_nuclei += 1

    # Convert lists to NumPy arrays
    inst_type_array = np.array(inst_type, dtype=np.int32)
    inst_uiud_array = np.array(inst_uiud, dtype=np.int32)

    # Create the annotation dictionary
    annotations = {
        'inst_map': inst_map,
        'type_map': type_map,
        'inst_type': inst_type_array,
        'inst_uiud': inst_uiud_array
    }

    # Save to a .mat file
    output_file = os.path.splitext(file_name)[0] + '.mat'
    scipy.io.savemat(output_file, annotations)

    print(f"\nResults for {file_name}:")
    print(f"Total features in GeoJSON: {total_features}")
    print(f"Valid unique nuclei processed: {valid_nuclei}")
    print(f"Duplicate nuclei skipped: {duplicate_nuclei}")
    print(f"Invalid or error nuclei skipped: {skipped_nuclei}")
    print(f"inst_map shape: {inst_map.shape}")
    print(f"type_map shape: {inst_map.shape}")
    print(f"inst_type shape: {inst_type_array.shape}")
    print(f"inst_uiud shape: {inst_uiud_array.shape}")
    print(f"Unique values in inst_map: {np.unique(inst_map)}")
    print(f"Unique values in type_map: {np.unique(type_map)}")

# Define the mapping from string to number
string_to_number = {
    "Tumor": 1,
    "Lymphocyte": 2,
    "Macrophage": 3,
    "Fibroblast": 4,
    "Vascular": 5,
    "nolabel": 6,
    "non-neoplastic": 7
}

# Define the files and their corresponding image shapes
files_and_shapes = [
    ("B-8.geojson", (4972, 4972)),
    ("D-3.geojson", (2633, 2821)),
    ("L-2.geojson", (4972, 4972)),
    ("L-8.geojson", (5095, 5010)),
    ("P-1.geojson", (4972, 4972))
]

# Process each file
for file_name, image_shape in files_and_shapes:
    process_geojson(file_name, image_shape)

Processing B-8.geojson: 100%|██████████| 13348/13348 [02:24<00:00, 92.17it/s]  



Results for B-8.geojson:
Total features in GeoJSON: 13348
Valid unique nuclei processed: 6680
Duplicate nuclei skipped: 6668
Invalid or error nuclei skipped: 0
inst_map shape: (4972, 4972)
type_map shape: (4972, 4972)
inst_type shape: (6680, 1)
inst_uiud shape: (6680, 1)
Unique values in inst_map: [   0    1    2 ... 6678 6679 6680]
Unique values in type_map: [0 1 2 3 4 5 6 7]


Processing D-3.geojson: 100%|██████████| 6211/6211 [00:37<00:00, 166.68it/s]



Results for D-3.geojson:
Total features in GeoJSON: 6211
Valid unique nuclei processed: 6211
Duplicate nuclei skipped: 0
Invalid or error nuclei skipped: 0
inst_map shape: (2633, 2821)
type_map shape: (2633, 2821)
inst_type shape: (6211, 1)
inst_uiud shape: (6211, 1)
Unique values in inst_map: [   0    1    2 ... 6209 6210 6211]
Unique values in type_map: [0 1 2 3 4 5 6 7]


Processing L-2.geojson: 100%|██████████| 5276/5276 [01:20<00:00, 65.84it/s]



Results for L-2.geojson:
Total features in GeoJSON: 5276
Valid unique nuclei processed: 5276
Duplicate nuclei skipped: 0
Invalid or error nuclei skipped: 0
inst_map shape: (4972, 4972)
type_map shape: (4972, 4972)
inst_type shape: (5276, 1)
inst_uiud shape: (5276, 1)
Unique values in inst_map: [   0    1    2 ... 5274 5275 5276]
Unique values in type_map: [0 1 2 3 4 5 6 7]


Processing L-8.geojson: 100%|██████████| 7746/7746 [03:00<00:00, 42.82it/s]



Results for L-8.geojson:
Total features in GeoJSON: 7746
Valid unique nuclei processed: 7746
Duplicate nuclei skipped: 0
Invalid or error nuclei skipped: 0
inst_map shape: (5095, 5010)
type_map shape: (5095, 5010)
inst_type shape: (7746, 1)
inst_uiud shape: (7746, 1)
Unique values in inst_map: [   0    1    2 ... 7744 7745 7746]
Unique values in type_map: [0 1 2 3 4 5 6 7]


Processing P-1.geojson: 100%|██████████| 7080/7080 [01:54<00:00, 62.02it/s]



Results for P-1.geojson:
Total features in GeoJSON: 7080
Valid unique nuclei processed: 7080
Duplicate nuclei skipped: 0
Invalid or error nuclei skipped: 0
inst_map shape: (4972, 4972)
type_map shape: (4972, 4972)
inst_type shape: (7080, 1)
inst_uiud shape: (7080, 1)
Unique values in inst_map: [   0    1    2 ... 7078 7079 7080]
Unique values in type_map: [0 1 2 3 4 5 6 7]


In [ ]:
import scipy.io
import numpy as np
import os

def analyze_mat_file(file_path):
    print(f"\nAnalyzing {file_path}:")

    # Load the .mat file
    mat_contents = scipy.io.loadmat(file_path)

    # Print each variable's name and its contents
    for key, value in mat_contents.items():
        if not key.startswith('__'):  # This avoids internal metadata fields
            print(f"Variable: {key}")
            print(f"Shape: {value.shape}")

            if key == 'inst_map':
                print(f"Unique values: {np.unique(value)}")
                print(f"Min value: {np.min(value)}")
                print(f"Max value: {np.max(value)}")
            elif key == 'type_map':
                print(f"Unique values: {np.unique(value)}")
                print(f"Min value: {np.min(value)}")
                print(f"Max value: {np.max(value)}")
            elif key in ['inst_type', 'inst_uiud']:
                print(f"Unique values: {np.unique(value)}")
                print(f"Contents: {value.flatten()[:10]}...")  # Print first 10 elements

            print()  # Add a blank line for readability

# List of .mat files to analyze
mat_files = ['B-8.mat', 'D-3.mat', 'L-2.mat', 'L-8.mat', 'P-1.mat']

# Process each file
for mat_file in mat_files:
    if os.path.exists(mat_file):
        analyze_mat_file(mat_file)
    else:
        print(f"File {mat_file} not found. Skipping.")


Analyzing B-8.mat:
Variable: inst_map
Shape: (4972, 4972)
Unique values: [   0    1    2 ... 6678 6679 6680]
Min value: 0
Max value: 6680

Variable: type_map
Shape: (4972, 4972)
Unique values: [0 1 2 3 4 5 6 7]
Min value: 0
Max value: 7

Variable: inst_type
Shape: (6680, 1)
Unique values: [1 2 3 4 5 6 7]
Contents: [1 1 1 6 1 1 1 1 1 2]...

Variable: inst_uiud
Shape: (6680, 1)
Unique values: [   1    2    3 ... 6678 6679 6680]
Contents: [ 1  2  3  4  5  6  7  8  9 10]...


Analyzing D-3.mat:
Variable: inst_map
Shape: (2633, 2821)
Unique values: [   0    1    2 ... 6209 6210 6211]
Min value: 0
Max value: 6211

Variable: type_map
Shape: (2633, 2821)
Unique values: [0 1 2 3 4 5 6 7]
Min value: 0
Max value: 7

Variable: inst_type
Shape: (6211, 1)
Unique values: [1 2 3 4 5 6 7]
Contents: [6 6 1 6 1 6 6 6 6 6]...

Variable: inst_uiud
Shape: (6211, 1)
Unique values: [   1    2    3 ... 6209 6210 6211]
Contents: [ 1  2  3  4  5  6  7  8  9 10]...


Analyzing L-2.mat:
Variable: inst_map
Shape: 

In [ ]:
from google.colab import files

mat_files = ['B-8.mat', 'D-3.mat', 'L-2.mat', 'L-8.mat', 'P-1.mat']

# Download each file
for file in mat_files:
    files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!python hovernet/extract_patches.py

  0% 0/5 [00:00<?, ?it/s]Original img shape: (4972, 4972, 3)
Original ann shape: (4972, 4972, 2)
Adjusted img shape: (4972, 4972, 3)
Adjusted ann shape: (4972, 4972, 2)

  0% 0/961 [00:00<?, ?it/s]
  0% 2/961 [00:00<00:49, 19.41it/s]
  0% 4/961 [00:00<00:49, 19.49it/s]
  1% 6/961 [00:00<00:48, 19.60it/s]
  1% 8/961 [00:00<00:48, 19.73it/s]
  1% 10/961 [00:00<00:48, 19.64it/s]
  1% 12/961 [00:00<00:48, 19.53it/s]
  1% 14/961 [00:00<00:48, 19.52it/s]
  2% 16/961 [00:00<00:48, 19.44it/s]
  2% 18/961 [00:00<00:48, 19.36it/s]
  2% 20/961 [00:01<00:48, 19.33it/s]
  2% 22/961 [00:01<00:48, 19.36it/s]
  2% 24/961 [00:01<00:48, 19.34it/s]
  3% 26/961 [00:01<00:48, 19.37it/s]
  3% 28/961 [00:01<00:47, 19.49it/s]
  3% 30/961 [00:01<00:47, 19.54it/s]
  3% 32/961 [00:01<00:47, 19.51it/s]
  4% 34/961 [00:01<00:47, 19.52it/s]
  4% 36/961 [00:01<00:47, 19.53it/s]
  4% 38/961 [00:01<00:47, 19.57it/s]
  4% 40/961 [00:02<00:47, 19.47it/s]
  4% 42/961 [00:02<00:47, 19.43it/s]
  5% 44/961 [00:02<00:47, 19.

If your NumPy is deprecated, you need to go to /usr/local/lib/python3.10/dist-packages/imgaug/augmenters/meta.py and change line 3368 from np.bool to np.bool_

replacing lines 213-214 w/

aligned_shape = np.min([
    imgs.shape[1:3],
    true_np.shape[1:3],
    pred_np.shape[1:3]
], axis=0)

In [ ]:
!python hovernet/run_train.py --gpu=0 #--view='train'


Detect #GPUS: 1
Using manual seed: 10
Dataset train: 4181
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Dataset valid: 4181
Model `net` pretrained path: /content/hovernet/ImageNet-ResNet50-Preact_pytorch.tar
Missing Variables: 
 ['conv_bot.weight', 'decoder.tp.u3.conva.weight', 'decoder.tp.u3.dense.units.0.preact_bna/bn.weight', 'decoder.tp.u3.dense.units.0.preact_bna/bn.bias', 'decoder.tp.u3.dense.units.0.preact_bna/bn.running_mean', 'decoder.tp.u3.dense.units.0.preact_bna/bn.running_var', 'decoder.tp.u3.dense.units.0.conv1.weight', 'decoder.tp.u3.dense.units.0.c

base.py line 68; strict = False



In [ ]:
import numpy as np
data = np.load("/content/hovernet/dataset/training_data/pannuke/pannuke/train/540x540_164x164/B-8_000.npy")
print(data.shape)
print(data[..., -1])
print(np.unique(data[..., -1]))

(540, 540, 5)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[0]


In [ ]:
import numpy as np
data = np.load("/content/hovernet/dataset/training_data/pannuke/pannuke/train/540x540_164x164/B-8_560.npy")
print(data.shape)
print(data[..., -2])
print(np.unique(data[..., -2]))

(540, 540, 5)
[[   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0 3690]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ... 4676 4676 4676]]
[   0 3634 3645 3654 3690 3692 3699 3705 3714 3729 3749 3757 3759 3764
 3784 3790 3792 3796 3803 3811 3820 3826 3831 3835 3836 3837 3838 3848
 3857 3859 3867 3872 3873 3896 3897 3906 3907 3911 3915 3924 3934 3938
 3945 3980 3985 3993 3994 4008 4014 4017 4023 4027 4032 4035 4036 4050
 4054 4064 4070 4090 4100 4114 4135 4143 4158 4163 4170 4185 4215 4218
 4260 4261 4262 4268 4274 4279 4280 4281 4291 4311 4316 4317 4323 4342
 4357 4360 4372 4379 4383 4404 4423 4432 4433 4435 4440 4446 4458 4477
 4492 4497 4502 4510 4528 4544 4558 4562 4567 4570 4571 4573 4596 4607
 4613 4614 4624 4630 4631 4637 4642 4643 4664 4671 4674 4676]


In [ ]:
!python hovernet/run_train.py --gpu=0 #--view='train'

Detect #GPUS: 1
Using manual seed: 10
Dataset train: 4181
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Dataset valid: 4181
Model `net` pretrained path: /content/hovernet/ImageNet-ResNet50-Preact_pytorch.tar
Missing Variables: 
 ['conv_bot.weight', 'decoder.tp.u3.conva.weight', 'decoder.tp.u3.dense.units.0.preact_bna/bn.weight', 'decoder.tp.u3.dense.units.0.preact_bna/bn.bias', 'decoder.tp.u3.dense.units.0.preact_bna/bn.running_mean', 'decoder.tp.u3.dense.units.0.preact_bna/bn.running_var', 'decoder.tp.u3.dense.units.0.conv1.weight', 'decoder.tp.u3.dense.units.0.c